# Objectif : faire benchmark avec la metric self check gpt

difficultés :
- regarder les performance sur les metrics pour des texte en fonction de la taille de la context window

In [1]:
import torch
assert torch.cuda.is_available()

In [2]:
all((1, 2, 3))

True

In [2]:
a = torch.tensor([1]).to("cuda")
del a

In [3]:
%load_ext autoreload
%autoreload 2
import os, sys, logging, random, asyncio, uvicorn, time
from typing import Union
import logging

import os
import sys
import json
import time
import re
from contextlib import asynccontextmanager
from pathlib import Path
from datetime import datetime
from collections import Counter
from typing import Dict, List
from urllib.parse import urlparse, urlsplit

from fastapi import FastAPI, HTTPException, status, Security, UploadFile
from fastapi.security.api_key import APIKeyHeader
from fastapi.responses import FileResponse, RedirectResponse
from fastapi.middleware.cors import CORSMiddleware
from langchain.chains.question_answering import load_qa_chain
from langchain_community.vectorstores.chroma import Chroma
from langchain_openai import ChatOpenAI
from langchain_community.embeddings import HuggingFaceEmbeddings
from langchain_openai import ChatOpenAI

sys.path.append("../src")
from selfcheck import SelfCheckGPT


[nltk_data] Downloading package punkt to /home/alex/nltk_data...
[nltk_data]   Package punkt is already up-to-date!


In [4]:
# grab a llm
OPENAI_API_BASE = os.environ["OPENAI_API_BASE"]
OPENAI_API_KEY = os.environ["OPENAI_API_KEY"]

llm = ChatOpenAI(
    api_key=OPENAI_API_KEY,
    openai_api_base=OPENAI_API_BASE,
    temperature=0,
    model="mixtral",
)


# Get sumup

In [5]:
# poetry add datasets
from datasets import load_dataset

dataset = load_dataset("ZhongshengWang/Alpaca-cnn-dailymail")

In [6]:
text_iterator = dataset["train"]["input"]

In [7]:
(text_iterator[0])

'It\'s official: U.S. President Barack Obama wants lawmakers to weigh in on whether to use military force in Syria. Obama sent a letter to the heads of the House and Senate on Saturday night, hours after announcing that he believes military action against Syrian targets is the right step to take over the alleged use of chemical weapons. The proposed legislation from Obama asks Congress to approve the use of military force "to deter, disrupt, prevent and degrade the potential for future uses of chemical weapons or other weapons of mass destruction." It\'s a step that is set to turn an international crisis into a fierce domestic political battle. There are key questions looming over the debate: What did U.N. weapons inspectors find in Syria? What happens if Congress votes no? And how will the Syrian government react? In a televised address from the White House Rose Garden earlier Saturday, the president said he would take his case to Congress, not because he has to -- but because he want

In [8]:
dataset

DatasetDict({
    train: Dataset({
        features: ['input', 'instruction', 'output'],
        num_rows: 574226
    })
    validation: Dataset({
        features: ['input', 'instruction', 'output'],
        num_rows: 26736
    })
    test: Dataset({
        features: ['input', 'instruction', 'output'],
        num_rows: 22980
    })
})

In [9]:
dataset["train"]["input"][2]
input_, instruction_, output_ = dataset["train"]["input"], dataset["train"]["instruction"], dataset["train"]["output"]

In [10]:
k = 0
input_[k]

'It\'s official: U.S. President Barack Obama wants lawmakers to weigh in on whether to use military force in Syria. Obama sent a letter to the heads of the House and Senate on Saturday night, hours after announcing that he believes military action against Syrian targets is the right step to take over the alleged use of chemical weapons. The proposed legislation from Obama asks Congress to approve the use of military force "to deter, disrupt, prevent and degrade the potential for future uses of chemical weapons or other weapons of mass destruction." It\'s a step that is set to turn an international crisis into a fierce domestic political battle. There are key questions looming over the debate: What did U.N. weapons inspectors find in Syria? What happens if Congress votes no? And how will the Syrian government react? In a televised address from the White House Rose Garden earlier Saturday, the president said he would take his case to Congress, not because he has to -- but because he want

In [11]:
len(input_[k])

9396

In [12]:
llm.get_num_tokens(input_[k])

1858

In [13]:
len(input_)

574226

In [14]:
from tqdm import tqdm
import statistics
res = [llm.get_num_tokens(j) for j in tqdm(input_[:10_000])]

  0%|          | 0/10000 [00:00<?, ?it/s]

100%|██████████| 10000/10000 [00:05<00:00, 1677.32it/s]


In [15]:
statistics.mean(res)

824.7222

In [34]:
from summary_chain import summarize_chain_builder, EmbeddingModel

embeddings = HuggingFaceEmbeddings(
    model_name="OrdalieTech/Solon-embeddings-base-0.1"
)  # plus de 13 min

if type(embeddings).__name__ == "HuggingFaceEmbeddings":
    model_class = "HuggingFaceEmbeddings"
else:
    model_class = "hugging_hub"

embedding_model = EmbeddingModel(embeddings, model_class)

custom_chain = summarize_chain_builder(
    method="text_rank", embedding_model=embedding_model, llm=llm
)

In [28]:
import httpx
import urllib.parse
text = text_iterator[0]

if 0:
    params = {'text': text}
    r = httpx.get('http://127.0.0.1:8000/text_summary?'+urllib.parse.urlencode(params), timeout=30)
    sumup = r.text
    assert len(text) > len(sumup)
else:
    sumup = custom_chain.invoke(text)


In [29]:
text

'It\'s official: U.S. President Barack Obama wants lawmakers to weigh in on whether to use military force in Syria. Obama sent a letter to the heads of the House and Senate on Saturday night, hours after announcing that he believes military action against Syrian targets is the right step to take over the alleged use of chemical weapons. The proposed legislation from Obama asks Congress to approve the use of military force "to deter, disrupt, prevent and degrade the potential for future uses of chemical weapons or other weapons of mass destruction." It\'s a step that is set to turn an international crisis into a fierce domestic political battle. There are key questions looming over the debate: What did U.N. weapons inspectors find in Syria? What happens if Congress votes no? And how will the Syrian government react? In a televised address from the White House Rose Garden earlier Saturday, the president said he would take his case to Congress, not because he has to -- but because he want

In [30]:
sumup

' U.S. President Barack Obama has requested lawmakers to consider the use of military force in Syria, as reported in a letter sent to congressional leaders on Saturday night. The decision comes after Obama announced his belief that striking Syrian targets is the appropriate response to the alleged use of chemical weapons by the Syrian government. The United Nations inspectors have left Syria and are currently analyzing evidence to determine if chemical weapons were used in an attack in a Damascus suburb earlier last week. Obama has proposed limited military action against Syrian President Bashar al-Assad, stating that the use of chemical weapons is "an assault on human dignity." The White House has emphasized the need for the international community to deliver a clear message to the Assad regime and others who may consider using chemical weapons that such actions are unacceptable. While some global leaders have expressed support, British Prime Minister David Cameron\'s attempt to gain 

# Eval sumup

In [16]:
import shelve
with shelve.open(".assup") as db:
    pass
sumup = Path("./assup.txt").read_text()
#Path("./assup.txt").write_text(sumup)
sumup

' U.S. President Barack Obama has requested lawmakers to consider the use of military force in Syria, as reported in a letter sent to congressional leaders on Saturday night. The decision comes after Obama announced his belief that striking Syrian targets is the appropriate response to the alleged use of chemical weapons by the Syrian government. The United Nations inspectors have left Syria and are currently analyzing evidence to determine if chemical weapons were used in an attack in a Damascus suburb earlier last week. Obama has proposed limited military action against Syrian President Bashar al-Assad, stating that the use of chemical weapons is "an assault on human dignity." The White House has emphasized the need for the international community to deliver a clear message to the Assad regime and others who may consider using chemical weapons that such actions are unacceptable. While some global leaders have expressed support, British Prime Minister David Cameron\'s attempt to gain 

In [17]:
text = text_iterator[0]


In [18]:
os.environ["CUDA_LAUNCH_BLOCKING"]="1"
os.environ["TORCH_USE_CUDA_DSA"]="1"

In [19]:
print(sumup)

 U.S. President Barack Obama has requested lawmakers to consider the use of military force in Syria, as reported in a letter sent to congressional leaders on Saturday night. The decision comes after Obama announced his belief that striking Syrian targets is the appropriate response to the alleged use of chemical weapons by the Syrian government. The United Nations inspectors have left Syria and are currently analyzing evidence to determine if chemical weapons were used in an attack in a Damascus suburb earlier last week. Obama has proposed limited military action against Syrian President Bashar al-Assad, stating that the use of chemical weapons is "an assault on human dignity." The White House has emphasized the need for the international community to deliver a clear message to the Assad regime and others who may consider using chemical weapons that such actions are unacceptable. While some global leaders have expressed support, British Prime Minister David Cameron's attempt to gain ba

In [20]:
len(sumup)

1438

In [26]:
from langchain.docstore.document import Document

from selfcheck import selfcheck

documents = [Document(page_content=text)]

metric = selfcheck(llm=llm, docs=documents, summarize_to_eval="Il faut être optimiste")


In [27]:
metric

0.0

In [28]:
# regarder les stats de la metric que le dataset de donné
control_metrics = []
for i, (current_input, current_output) in enumerate(zip(tqdm(input_[:10]), output_)):
    documents = [Document(page_content=current_input)]

    metric = selfcheck(llm=llm, docs=documents, summarize_to_eval=current_output)
    control_metrics.append(metric)


100%|██████████| 50/50 [08:48<00:00, 10.57s/it]


In [29]:
statistics.mean(control_metrics)

0.8186666666666667

81.8% sur les resumé dispo sur hugging face. Combian avec le TextRank ?

In [45]:
metrics =  []
for text_to_sumup in tqdm(input_[:10]):
    sumup = custom_chain.invoke(text_to_sumup)
    documents = [Document(page_content=text_to_sumup)]
    metric = selfcheck(llm=llm, docs=documents, summarize_to_eval=sumup)
    metrics.append(metric)

  0%|          | 0/10 [00:00<?, ?it/s]

100%|██████████| 10/10 [05:12<00:00, 31.23s/it]


In [40]:
current_output

"Ruben Navarrette: Nine Dreamer activists were in federal immigration detention in Arizona .\nThey'd tried to re-enter the U.S. asking for humanitarian parole or asylum and were jailed .\nHe says they have been freed, granted immigration hearing, drew support from Congress .\nNavarrette: DREAMers sometimes have look-at-me activism, but this request is valid ."

In [41]:
sumup

' Edward Nkoloso, a Zambian grade-school science teacher, established an academy for space training on the outskirts of Lusaka in the 1960s to prepare for an interstellar journey to Mars. The crew included a specially trained spacegirl named Matha Mwambwa, two cats, and a missionary. In 2010, Spanish photographer Cristina De Middel learned about this unofficial Zambian space program while searching for intriguing stories for a new project. She created a visual narrative of Nkoloso\'s endeavor through her self-published book, "The Afronauts," blending fact and fiction to spark curiosity and challenge the documentary value of photography. De Middel has received invitations from African space programs and has exhibited her work in South Africa and Senegal. She considers her images a reflection of her own cliché about Africa, aiming to raise awareness and discuss the continent from a different perspective. The story highlights Nkoloso\'s passion for the universe and his dream of achieving 

In [17]:
from langchain.tools import WikipediaQueryRun
from langchain_community.utilities import WikipediaAPIWrapper
wikipedia = WikipediaQueryRun(api_wrapper=WikipediaAPIWrapper())

In [18]:
wikipedia.run("Albert Camus")

'Page: Albert Camus\nSummary: Albert Camus ( kam-OO; French: [albɛʁ kamy] ; 7 November 1913 – 4 January 1960) was a French philosopher, author, dramatist, journalist, world federalist, and political activist. He was the recipient of the 1957 Nobel Prize in Literature at the age of 44, the second-youngest recipient in history. His works include The Stranger, The Plague, The Myth of Sisyphus, The Fall, and The Rebel.\nCamus was born in Algeria during the French colonization, to pied-noir parents. He spent his childhood in a poor neighbourhood and later studied philosophy at the University of Algiers. He was in Paris when the Germans invaded France during World War II in 1940. Camus tried to flee but finally joined the French Resistance where he served as editor-in-chief at Combat, an outlawed newspaper. After the war, he was a celebrity figure and gave many lectures around the world. He married twice but had many extramarital affairs. Camus was politically active; he was part of the left

In [16]:
import wikipedia
wikipedia.summary("Albert camus")

PageError: Page id "albert cams" does not match any pages. Try another id!

In [19]:
import wikiscraper as ws
ws.lang("fr")
result = ws.searchBySlug("Albert_Camus")

In [24]:
result.getContentsTable()

['Début',
 '1Biographie',
 '2Philosophie',
 '3Distinctions',
 '4Postérité',
 '5Œuvres',
 '6Parutions posthumes',
 '7Notes et références',
 '8Voir aussi']

In [30]:
from datasets import load_dataset
wiki = load_dataset("wikipedia", language="fr", date="20220301", trust_remote_code=True)

Generating train split:   0%|          | 0/2402095 [00:00<?, ? examples/s]

In [50]:
def grab_content(url):
    idx = wiki["train"]["url"].index(url)     
    return wiki["train"]["text"][idx]

wiki["train"]["url"]#["https://fr.wikipedia.org/wiki/Albert_Camus"]
c = grab_content("https://fr.wikipedia.org/wiki/Albert%20Camus")
len(c)

49422

In [54]:
content = (("albertcamus", "https://fr.wikipedia.org/wiki/Albert%20Camus"),
           ("georgeorwell", "https://fr.wikipedia.org/wiki/George%20Orwell"),
           ("julesverne", "https://fr.wikipedia.org/wiki/Jules%20Verne"),
           ("victorhugo", "https://fr.wikipedia.org/wiki/Victor%20Hugo"),
           ("beethoven", "https://fr.wikipedia.org/wiki/Ludwig%20van%20Beethoven"))
from pathlib import Path
for name, url in content:
    (Path("/home/alex/CODE/abrege/texts") / f"{name}.txt").write_text(grab_content(url))

In [55]:
wiki["train"]

Dataset({
    features: ['id', 'url', 'title', 'text'],
    num_rows: 2402095
})

In [49]:
wiki["train"]["text"][12]

'Ada est un langage de programmation orienté objet dont les premières versions remontent au début des années 1980.\n\nPrésentation \nAda est un langage de programmation conçu par l’équipe de CII-Honeywell Bull dirigée par Jean Ichbiah en réponse à un cahier des charges établi par le département de la Défense des États-Unis (DoD). Les premiers travaux autour du projet ont commencé à la fin des années 1970 et dès 1979 un preliminary ADA reference manual est publié dans la revue du Special Interest Group on Programming Languages de l\'ACM. La première version validée du projet sort au début des années 1980  sous le nom d\'Ada 83.\n\nPour développer le langage, l\'équipe de Jean Ichbiah s\'est inspirée de son précédent langage LIS (Langage d\'Implémentation de Systèmes), conçu à la CII pour permettre le développement de systèmes d\'exploitation portables (à 95 %, disait Ichbiah).\n\nAda 83 a été ensuite repris et amélioré au milieu des années 1990 pour donner Ada 95, le second langage obje

In [47]:
wiki["train"]["url"].index("https://fr.wikipedia.org/wiki/Albert%20Camus")

237665

In [46]:
[f for f in wiki["train"]["url"] if "camus" in f.lower()]

['https://fr.wikipedia.org/wiki/Louis-Camus%20Destouches',
 'https://fr.wikipedia.org/wiki/Camus',
 'https://fr.wikipedia.org/wiki/Jean-Yves%20Camus',
 'https://fr.wikipedia.org/wiki/Michel%20Camus',
 'https://fr.wikipedia.org/wiki/Marcel%20Camus',
 'https://fr.wikipedia.org/wiki/Camus%20%28Saint%20Seiya%29',
 'https://fr.wikipedia.org/wiki/Perrenot%20Camus%20de%20Marcilly',
 'https://fr.wikipedia.org/wiki/Geoffroy%20Camus%20de%20Pontcarr%C3%A9',
 'https://fr.wikipedia.org/wiki/Jean-Pierre%20Camus',
 'https://fr.wikipedia.org/wiki/Nicolas%20Pierre%20Camus%20de%20Pontcarr%C3%A9',
 'https://fr.wikipedia.org/wiki/Geoffroy%20Mac%C3%A9%20Camus%20de%20Pontcarr%C3%A9',
 'https://fr.wikipedia.org/wiki/Charles%20%C3%89tienne%20Louis%20Camus',
 'https://fr.wikipedia.org/wiki/Jean-Claude%20Camus',
 'https://fr.wikipedia.org/wiki/Albert%20Camus',
 'https://fr.wikipedia.org/wiki/AfricaMuseum',
 'https://fr.wikipedia.org/wiki/Nicolas%20Le%20Camus%20de%20M%C3%A9zi%C3%A8res',
 'https://fr.wikipedia.or

In [21]:
result.getAbstract()

AttributeError: 'NoneType' object has no attribute 'nextSibling'

In [14]:
result.getSection("Géographie")

["Paris se situe au cœur d'un vaste bassin sédimentaire aux sols fertiles et au climat tempéré, le bassin parisien, sur une boucle de la Seine, entre les confluents de celle-ci avec la Marne et l'Oise.\n",
 'Les départements limitrophes sont les Hauts-de-Seine, la Seine-Saint-Denis et le Val de Marne.\n',
 '\n',
 '\nLes limites communales de Paris et celles de ses communes adjacentes.\n\n',
 '\n\nCommunes limitrophes de Paris\n\n\nClichy,Levallois-Perret,Neuilly-sur-Seine\n\nSaint-Ouen-sur-Seine, Saint-Denis, Aubervilliers\n\nPantin,Le Pré-Saint-Gervais,Les Lilas\n\n\nPuteaux,Suresnes,Saint-Cloud\n\n\n\nBagnolet, Montreuil, Saint-Mandé, Vincennes, Fontenay-sous-Bois, Nogent-sur-Marne\n\n\nBoulogne-Billancourt, Issy-les-Moulineaux, Vanves\n\nMalakoff, Montrouge, Gentilly, Le Kremlin-Bicêtre\n\nJoinville-le-Pont, Saint-Maurice, Charenton-le-Pont, Ivry-sur-Seine\n',
 'Selon la géographie des régions naturelles de France, la ville de Paris se situe entre le Pays de France (rive droite) et 

In [ ]:
import torch

In [ ]:
torch.tensor([1]).to("cuda")

RuntimeError: CUDA error: unspecified launch failure
CUDA kernel errors might be asynchronously reported at some other API call, so the stacktrace below might be incorrect.
For debugging consider passing CUDA_LAUNCH_BLOCKING=1.
Compile with `TORCH_USE_CUDA_DSA` to enable device-side assertions.


In [ ]:
sys.path.append("./src")
from summary_chain import summarize_chain_builder, EmbeddingModel

# on va tester summary_chain avec selfcheck